## Compute tick count for normalized sources (FUTURES) - Python

### Overview

TODO

### Inputs/outputs

TODO

### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* TopologiesService: to directly retrieve ticks objects from the server.

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.topology.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package

***

# Run market activity sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib

In [ ]:
pip install -i https://test.pypi.org/simple/ systemathics.apis==0.9.27

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import date
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.topology.v1.topologies_pb2 as topologies
import systemathics.apis.services.topology.v1.topologies_pb2_grpc as topologies_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker + sources
tickerexchange_array = [['AAPL', 'XNGS',564],
                        ['MSFT', 'BATS',564],
                        ['AMZN', 'XNGS',564],
                        ['AAPL', 'BATS',729],
                        ['MSFT', 'XNGS',729],
                        ['AMZN', 'BATS',729],
                        ['ASML', 'XAMS', 787],
                        ['ABI', 'XBRU',787],
                        ['UNA', 'XAMS',787],
                        ['RDSA', 'XAMS',787],
                        ['ARGX', 'XBRU',787],
                        ['MC', 'XPAR',787],
                        ['SAN', 'CHIX',787],
                        ['FP', 'XPAR',787],
                        ['JMT', 'XLIS',787],
                        ['EDPR', 'XLIS',787],
                        ['EDP', 'CHIX',787],
                        ['ASML', 'CHIX',794],
                        ['ABI', 'CHIX',794],
                        ['KBC', 'CHIX',794],
                        ['KBC', 'XBRU',794],
                        ['ARGX', 'CHIX',794],
                        ['MC', 'CHIX',794],
                        ['SAN', 'CHIX',794],
                        ['JMT', 'CHIX',794],
                        ['EDPR', 'CHIX',794],
                        ['EDP', 'XLIS',794]
                       ]
length = len(tickerexchange_array)

#### 3.2 Topology parmeters

In [ ]:
# set topology time granularity (daily, weekly...)
granularity = topologies.TOPOLOGY_GRANULARITY_DAILY

# set level: Trades or Trades and Book
level = level.LEVEL_TRADES_AND_BOOK

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
def get_topologies_request(ticker, exchange, granularity, level):
    request = topologies.TopologiesRequest(identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
                                           granularity = granularity,
                                           level = level)
    return request

In [ ]:
# Process all topologies requests
credentials = grpc.ssl_channel_credentials()
equities_responses =[]
today = datetime.today()
      
# Iterate all exhange/ticker pairs
for i in range(length):
    with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:

        # instantiate the topologies service
        ticker = tickerexchange_array[i][0]
        exchange = tickerexchange_array[i][1]
        request = get_topologies_request(ticker, exchange, granularity, level)
        service = topologies_service.TopologiesServiceStub(channel)

        # process the topologies request
        response = service.Topologies(
            request=request, 
            metadata = [('authorization', token)]
        )
        
        #store
        equities_responses.append(response)
        
# get tick count data
print("Total asset requests: ", length)

In [ ]:
import csv

# Process all topologies responses
today = datetime.today()
filename = 'topologies_equities_{0:%Y%m%d}.csv'.format(today)

with open(filename, mode='w') as topologies_equities_file:
    topologies_equities_writer = csv.writer(topologies_equities_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row
    topologies_equities_writer.writerow(['Ticker', 'Exchange', 'Source', 'Entries' ,'Total_ticks', 'First_tick', 'Last_tick', 'Missing_days'])
            
    # Iterate all exhange/ticker pairs
    for i in range(length):

        response = equities_responses[i]    
        # instantiate the topologies service
        ticker = tickerexchange_array[i][0]
        exchange = tickerexchange_array[i][1]

        entries_count = len(response.entries)
        tick_counts = sum([entry.ticks_count for entry in response.entries])
        first_date = datetime(response.entries[0].begin.year, response.entries[0].begin.month, response.entries[0].begin.day)
        last_date = datetime(response.entries[-1].end.year, response.entries[-1].end.month, response.entries[-1].end.day)
        missing_days = (today- last_date).days
        source = tickerexchange_array[i][2]
        print("Total entries for {0}-{1} ({2}) \t: {3} \t| total ticks count: {4} \t | b: {5:%Y/%m/%d} - {6:%Y/%m/%d} \t| Missing days: {7}".format(ticker, exchange, source ,entries_count, tick_counts, first_date, last_date,missing_days))
        topologies_equities_writer.writerow([ticker,exchange, source, entries_count, tick_counts, '{0:%Y/%m/%d}'.format(first_date), '{0:%Y/%m/%d}'.format(last_date), missing_days])

### Step 4: Visualize data

#### 4.1 Plot tick count
The following code snippets diplay tick count's request results with the package of your choice.<br>*`matplotlib`* is used as per open source display package. Plot a simple line graph with the closing price and volume to easily spot spikes in trading as follows:

In [ ]:
import matplotlib.pyplot as plt

for i in range(length):
    counts = [entry.ticks_count for entry in equities_responses[i].entries]
    dates = [datetime(year=entry.begin.year,day=entry.begin.day, month=entry.begin.month) for entry in equities_responses[i].entries]
    
    # plot
    fig,ax = plt.subplots(1,1,figsize=(25,10))
    ax.bar(dates,counts)
    plt.xlabel("Date",size="20")
    plt.ylabel("Tick count",size="20")
    plt.title("Tick count for {0}-{1} | source: {2}".format(tickerexchange_array[i][0],tickerexchange_array[i][1], tickerexchange_array[i][2]),size="20")
    plt.show()